In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv("/kaggle/input/playground-series-s3e6/train.csv")
train.head()

,id,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom,price
0,0,34291,24,1,0,47,35693,2,1,2000,0,1,8,5196,369,0,3,3436795.2
1,1,95145,60,0,1,60,34773,1,4,2000,0,1,729,4496,277,0,6,9519958.0
2,2,92661,45,1,1,62,45457,4,8,2020,1,1,7473,8953,245,1,9,9276448.1
3,3,97184,99,0,0,59,15113,1,1,2000,0,1,6424,8522,256,1,9,9725732.2
4,4,61752,100,0,0,57,64245,8,4,2018,1,0,7151,2786,863,0,7,6181908.8


In [3]:
test = pd.read_csv("/kaggle/input/playground-series-s3e6/test.csv")
test.head()

,id,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom
0,22730,47580,89,0,1,8,54830,5,3,1995,0,0,6885,8181,241,0,8
1,22731,62083,38,0,0,87,8576,10,3,1994,1,1,4601,9237,393,1,4
2,22732,90499,75,1,1,37,62454,9,6,1997,0,1,7454,2680,305,0,2
3,22733,16354,47,1,1,9,9262,6,5,2019,1,1,705,5097,122,1,5
4,22734,67510,8,0,0,55,24112,3,7,2014,1,1,3715,7979,401,1,9


In [4]:
original_data = pd.read_csv("/kaggle/input/paris-housing-price-prediction/ParisHousing.csv")
original_data.head()

,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom,price
0,75523,3,0,1,63,9373,3,8,2005,0,1,4313,9005,956,0,7,7559081.5
1,80771,39,1,1,98,39381,8,6,2015,1,0,3653,2436,128,1,2,8085989.5
2,55712,58,0,1,19,34457,6,8,2021,0,0,2937,8852,135,1,9,5574642.1
3,32316,47,0,0,6,27939,10,4,2012,0,1,659,7141,359,0,3,3232561.2
4,70429,19,1,1,90,38045,3,7,1990,1,0,8435,2429,292,1,4,7055052.0


In [5]:
testid = test['id']

In [6]:
train = train.drop(['id'], axis=1)
test = test.drop(['id'], axis = 1)

In [7]:
# adding the source column
original_data['source'] = 0
train['source'] = 1
test['source'] = 1

# combining the datasets
train = pd.concat([train, original_data])
train.reset_index(inplace=True, drop=True)
train.shape

(32730, 18)

In [8]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold

In [9]:
train.columns

Index(['squareMeters', 'numberOfRooms', 'hasYard', 'hasPool', 'floors',
       'cityCode', 'cityPartRange', 'numPrevOwners', 'made', 'isNewBuilt',
       'hasStormProtector', 'basement', 'attic', 'garage', 'hasStorageRoom',
       'hasGuestRoom', 'price', 'source'],
      dtype='object')

In [10]:
train["ActualArea"] = train['squareMeters'] - train['basement'] - train['attic']-train['garage']
train['AreaPerHouse'] = train['squareMeters'] / train['numberOfRooms']
train['AreaPerRoom'] = train['ActualArea'] / train['numberOfRooms']
train['Extra'] = train['AreaPerHouse'] - train['AreaPerRoom']
train.head()

,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,...,attic,garage,hasStorageRoom,hasGuestRoom,price,source,ActualArea,AreaPerHouse,AreaPerRoom,Extra
0,34291,24,1,0,47,35693,2,1,2000,0,...,5196,369,0,3,3436795.2,1,28718,1428.791667,1196.583333,232.208333
1,95145,60,0,1,60,34773,1,4,2000,0,...,4496,277,0,6,9519958.0,1,89643,1585.750000,1494.050000,91.700000
2,92661,45,1,1,62,45457,4,8,2020,1,...,8953,245,1,9,9276448.1,1,75990,2059.133333,1688.666667,370.466667
3,97184,99,0,0,59,15113,1,1,2000,0,...,8522,256,1,9,9725732.2,1,81982,981.656566,828.101010,153.555556
4,61752,100,0,0,57,64245,8,4,2018,1,...,2786,863,0,7,6181908.8,1,50952,617.520000,509.520000,108.000000


In [11]:
test["ActualArea"] = test['squareMeters'] - test['basement'] - test['attic']-test['garage']
test['AreaPerHouse'] = test['squareMeters'] / test['numberOfRooms']
test['AreaPerRoom'] = test['ActualArea'] / test['numberOfRooms']
test['Extra'] = test['AreaPerHouse'] - test['AreaPerRoom']
test.head()

,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,...,basement,attic,garage,hasStorageRoom,hasGuestRoom,source,ActualArea,AreaPerHouse,AreaPerRoom,Extra
0,47580,89,0,1,8,54830,5,3,1995,0,...,6885,8181,241,0,8,1,32273,534.606742,362.617978,171.988764
1,62083,38,0,0,87,8576,10,3,1994,1,...,4601,9237,393,1,4,1,47852,1633.763158,1259.263158,374.500000
2,90499,75,1,1,37,62454,9,6,1997,0,...,7454,2680,305,0,2,1,80060,1206.653333,1067.466667,139.186667
3,16354,47,1,1,9,9262,6,5,2019,1,...,705,5097,122,1,5,1,10430,347.957447,221.914894,126.042553
4,67510,8,0,0,55,24112,3,7,2014,1,...,3715,7979,401,1,9,1,55415,8438.750000,6926.875000,1511.875000


In [12]:
Price = train['price']

In [13]:
cor = train.corr()
cor.to_csv("cor.csv")

In [14]:
train = train.drop(['ActualArea', 'AreaPerRoom'], axis = 1)
test = test.drop(['ActualArea', 'AreaPerRoom'], axis = 1)